In [1]:
import sys
sys.path.insert(0, '/home/llr/cms/wind/.local/lib/python2.7/site-packages/')

In [2]:
from bayes_opt import BayesianOptimization
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
import math

In [3]:
from subprocess import check_output

In [4]:
from trainlib.ConfigFileHandler import ConfigFileHandler

In [5]:
run_dir = "/data_CMS/cms/wind/180410_class_weights_Adam_smallbatch/input_columns_4/"
engine = "tree"
out_dir = "/data_CMS/cms/wind/prior_evaluator_2/"

In [10]:
evalcnt = 0

def punzi_target(ggH_prior, WHhadr_prior, ZHhadr_prior, WHlept_prior, ZHlept_prior, ZHMET_prior, ttHhadr_prior, ttHlept_prior):
    global evalcnt
    bin_dir = "/home/llr/cms/wind/cmssw/CMSSW_9_4_2/bin/slc6_amd64_gcc630/"
    cost_function_evaluator = "run_prior_evaluator"
    
    output = check_output([bin_dir + cost_function_evaluator, run_dir, out_dir, engine, str(ggH_prior), str(WHhadr_prior), 
              str(ZHhadr_prior), str(WHlept_prior), str(ZHlept_prior), str(ZHMET_prior), str(ttHhadr_prior), str(ttHlept_prior)])
    
    costval = 0.0

    for line in output.split('\n'):
        if "cost = " in line:
            costval = -float(line.replace("cost = ", ""))
            break
    
    # save the sampled point such that later they can be used as exploration points (if the need occurs)
               
    if math.isnan(costval):
        costval = -10.0
        
    confhandler = ConfigFileHandler()
    evaluations_path = out_dir + 'evaluations.txt'
    
    if os.path.exists(evaluations_path):
        confhandler.load_configuration(evaluations_path)
    
    section_name = 'evaluation_' + str(evalcnt)
    confhandler.new_section(section_name)
    confhandler.set_field(section_name, 'cost', str(costval))
    confhandler.set_field(section_name, 'ggH_prior', str(ggH_prior))
    confhandler.set_field(section_name, 'WHhadr_prior', str(WHhadr_prior))
    confhandler.set_field(section_name, 'ZHhadr_prior', str(ZHhadr_prior))
    confhandler.set_field(section_name, 'WHlept_prior', str(WHlept_prior))
    confhandler.set_field(section_name, 'ZHlept_prior', str(ZHlept_prior))
    confhandler.set_field(section_name, 'ZHMET_prior', str(ZHMET_prior))
    confhandler.set_field(section_name, 'ttHhadr_prior', str(ttHhadr_prior))
    confhandler.set_field(section_name, 'ttHlept_prior', str(ttHlept_prior))
    
    confhandler.save_configuration(evaluations_path)
         
    evalcnt += 1
 
    return costval

In [11]:
eps = 1e-3

In [25]:
bo = BayesianOptimization(punzi_target, {'ggH_prior': (1.0 + eps, 4.0), 'WHhadr_prior': (eps, 1.0 - eps), 
                                   'ZHhadr_prior': (eps, 1.0 - eps), 'WHlept_prior': (eps, 1.0 - eps),
                                   'ZHlept_prior': (eps, 1.0 - eps), 'ZHMET_prior': (eps, 1.0 - eps),
                                   'ttHhadr_prior': (eps, 1.0 - eps), 'ttHlept_prior': (eps, 1.0 - eps)})

In [14]:
# check if a file with previously evaluated points exists, if so, use them for initialization
confhandler = ConfigFileHandler()
evaluations_path = out_dir + 'evaluations.txt'

if os.path.exists(evaluations_path):
    confhandler.load_configuration(evaluations_path)

attempting to load configuration file from /data_CMS/cms/wind/prior_evaluator_2/evaluations.txt


In [20]:
ggH_priors_init = []
WHhadr_priors_init = []
ZHhadr_priors_init = []
WHlept_priors_init = []
ZHlept_priors_init = []
ZHMET_priors_init = []
ttHhadr_priors_init = []
ttHlept_priors_init = []
targets_init = []

for section_name in confhandler.get_sections():
    cur_section = confhandler.get_section(section_name)
    
    targets_init.append(float(cur_section['cost']))
    ggH_priors_init.append(float(cur_section['ggH_prior']))
    WHhadr_priors_init.append(float(cur_section['WHhadr_prior']))
    ZHhadr_priors_init.append(float(cur_section['ZHhadr_prior']))
    WHlept_priors_init.append(float(cur_section['WHlept_prior']))
    ZHlept_priors_init.append(float(cur_section['ZHlept_prior']))
    ZHMET_priors_init.append(float(cur_section['ZHMET_prior']))
    ttHhadr_priors_init.append(float(cur_section['ttHhadr_prior']))
    ttHlept_priors_init.append(float(cur_section['ttHlept_prior']))

In [22]:
init_dict = {'target': targets_init, 'ggH_prior': ggH_priors_init, 'WHhadr_prior': WHhadr_priors_init,
            'ZHhadr_prior': ZHhadr_priors_init, 'WHlept_prior': WHlept_priors_init, 'ZHlept_prior': ZHlept_priors_init,
            'ZHMET_prior': ZHMET_priors_init, 'ttHhadr_prior': ttHhadr_priors_init, 'ttHlept_prior': ttHlept_priors_init}

In [26]:
bo.initialize(init_dict)

In [27]:
bo.maximize(init_points = 6, n_iter = 0, acq = 'ucb', kappa = 3)

Initialization
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   WHhadr_prior |   WHlept_prior |   ZHMET_prior |   ZHhadr_prior |   ZHlept_prior |   ggH_prior |   ttHhadr_prior |   ttHlept_prior | 
attempting to load configuration file from /data_CMS/cms/wind/prior_evaluator_2/evaluations.txt
    1 | 00m00s |   32.17140 |         0.3297 |         0.3204 |        0.6831 |         0.0207 |         0.1640 |      3.3820 |          0.6545 |          0.0439 | 
attempting to load configuration file from /data_CMS/cms/wind/prior_evaluator_2/evaluations.txt
    2 | 00m00s |   21.34357 |         0.4926 |         0.7171 |        0.0974 |         0.3505 |         0.7937 |      2.3806 |          0.1140 |          0.7380 | 
    3 | 00m00s |   12.79807 |         0.4868 |         0.8979 |        0.0632 |         0.7755 |         0.1225 |      1.5232 |       

In [ ]:
for it in range(500):
    bo.maximize(init_points = 6, n_iter = 1, acq = 'ucb', kappa = 3)
    
    # evaluate the current maximum
    curval = bo.res['max']
    cost = curval['max_val']
    priors = curval['max_params']
    
    confhandler = ConfigFileHandler()
    confhandler.config.optionxform = str
    confhandler.new_section('Priors')
    confhandler.set_field('Priors', 'cost', str(cost))
    confhandler.set_field('Priors', 'VBF_prior', str(1.0))
        
    for key, val in priors.iteritems():
        confhandler.set_field('Priors', key, str(val))
        
    confhandler.save_configuration(out_dir + 'priors.txt')

Bayesian Optimization
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   WHhadr_prior |   WHlept_prior |   ZHMET_prior |   ZHhadr_prior |   ZHlept_prior |   ggH_prior |   ttHhadr_prior |   ttHlept_prior | 
   61 | 00m06s |    0.71976 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         0.0010 |      1.0000 |          0.0010 |          0.0010 | 
   62 | 00m06s |    0.58590 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          1.0000 | 
   63 | 00m07s |    0.08640 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         0.0010 |      1.0000 |          1.0000 |          1.0000 | 
   64 | 00m06s |    0.22026 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          1.0000 |         

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -2.46165992e-05]), 'nit': 4, 'funcalls': 47}
  " state: %s" % convergence_dict)


   92 | 00m30s |    0.33157 |         0.0010 |         0.0010 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.5698 |          0.7758 | 
   93 | 00m30s |    0.62557 |         0.2933 |         0.0010 |        1.0000 |         1.0000 |         0.0010 |      1.0000 |          0.0010 |          0.4983 | 
   94 | 00m31s |    0.30882 |         0.3510 |         0.5834 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          1.0000 | 
   95 | 00m29s |    0.64945 |         0.0010 |         0.2950 |        1.0000 |         0.7650 |         1.0000 |      1.0000 |          0.0010 |          0.2537 | 
   96 | 00m32s |    0.66601 |         0.2896 |         0.2934 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          0.0010 | 
   97 | 00m34s |    0.49665 |         0.2736 |         0.0010 |        1.0000 |         0.7451 |         1.0000 |      1.0000 |          0.3786 |          0.0010 | 
   98 | 00

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ -9.02500935e-05]), 'nit': 6, 'funcalls': 57}
  " state: %s" % convergence_dict)


  134 | 01m38s |    0.72716 |         0.1490 |         0.0010 |        1.0000 |         1.0000 |         0.3749 |      1.0000 |          0.0010 |          0.0010 | 
  135 | 01m40s |    0.74648 |         0.1228 |         0.0515 |        1.0000 |         0.9038 |         0.0025 |      1.0000 |          0.0010 |          0.0010 | 


...

  154 | 03m45s |    0.72932 |         0.0010 |         0.0010 |        1.0000 |         0.9211 |         0.0779 |      1.0000 |          0.0010 |          0.0010 | 
  155 | 04m05s |    0.74838 |         0.1143 |         0.1144 |        1.0000 |         0.8761 |         1.0000 |      1.0000 |          0.0010 |          0.0010 | 
  156 | 03m55s |    0.69455 |         0.0010 |         0.1649 |        1.0000 |         1.0000 |         0.1110 |      1.0000 |          0.0010 |          0.2308 | 


/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00011339]), 'nit': 6, 'funcalls': 51}
  " state: %s" % convergence_dict)


  157 | 04m22s |    0.72101 |         0.0010 |         0.1925 |        1.0000 |         1.0000 |         1.0000 |      1.0000 |          0.0010 |          0.0010 | 


/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-fmblme3/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([  9.29640346e-05]), 'nit': 6, 'funcalls': 49}
  " state: %s" % convergence_dict)


  158 | 04m20s |    0.72956 |         0.0010 |         0.1015 |        1.0000 |         1.0000 |         0.4509 |      1.0000 |          0.0010 |          0.0010 | 
  159 | 04m42s |    0.72210 |         0.0521 |         0.0010 |        1.0000 |         0.7768 |         0.1338 |      1.0000 |          0.0010 |          0.0010 | 
  160 | 04m22s |    0.72920 |         0.1981 |         0.1012 |        1.0000 |         1.0000 |         0.4615 |      1.0000 |          0.0010 |          0.0051 | 
